In [3]:
pip install tensorflow

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Python39\\Lib\\site-packages\\tensorflow\\python\\keras\\utils\\data_utils.py'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'C:\Python39\python.exe -m pip install --upgrade pip' command.



  Using cached tensorflow-2.12.0-cp39-cp39-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.12.0-cp39-cp39-win_amd64.whl (272.8 MB)


In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [3]:
# Paths to the train, test, and validation data folders
train_dir = 'split_data/train'
test_dir = 'split_data/test'
val_dir = 'split_data/val'

In [4]:
# Image dimensions
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)

# Batch size
batch_size = 32

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

Found 5536 images belonging to 15 classes.
Found 706 images belonging to 15 classes.
Found 686 images belonging to 15 classes.


In [7]:
# Model architecture
base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

In [9]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
# Train the model
history = model.fit(train_generator,
                    epochs=20,
                    validation_data=val_generator,
                    verbose=1)

In [13]:
# Evaluate the model on the test data
score = model.evaluate(test_generator, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.9439961910247803
Test accuracy: 0.5580736398696899


In [1]:
import numpy as np
from sklearn.metrics import label_ranking_average_precision_score, top_k_accuracy_score

# Predict on the test data
y_pred = model.predict(test_generator)

# Convert the generator to an array of labels
y_true = np.concatenate([test_generator.next()[1] for i in range(test_generator.samples // batch_size + 1)])

# Calculate LWLRAP
lwlrAP = label_ranking_average_precision_score(y_true, y_pred)

# Calculate Top-5 categorical accuracy
top5_acc = top_k_accuracy_score(y_true, y_pred, k=5)

print('LWLRAP:', lwlrAP)
print('Top-5 categorical accuracy:', top5_acc)


NameError: name 'model' is not defined